In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(r'../input/bank-customer-churn-modeling/Churn_Modelling.csv')
df.head(5)

In [ ]:
df = df.drop(['CustomerId','RowNumber','Surname'], axis = "columns")
df

In [ ]:
df.isnull().sum()

In [ ]:
import seaborn as sns

In [ ]:
sns.barplot(x = df.Exited, y = df.Geography)

In [ ]:
sns.scatterplot(x = df['EstimatedSalary'], y = df['Exited'], hue = df['CreditScore'])

In [ ]:
geo_dum = pd.get_dummies(df.Geography)
df = pd.concat([df,geo_dum],axis = "columns")

In [ ]:
gen_dum = pd.get_dummies(df.Gender)
df = pd.concat([df,gen_dum],axis = "columns")

In [ ]:
df.head(5)

In [ ]:
df = df.drop(['France','Geography','Gender','Male'],axis = "columns")
df.head(5)

In [ ]:
df.Age.describe()

In [ ]:
df = df.loc[(df.Age <= 80)]

In [ ]:
df.head(5)

In [ ]:
df.EstimatedSalary.describe()

In [ ]:
df

In [ ]:
Q1 , Q3 = df.EstimatedSalary.quantile([0.25, 0.85])
Q1, Q3

In [ ]:
df = df.loc[(df.EstimatedSalary >=  Q1) & (df.EstimatedSalary <= Q3)]
df

In [ ]:
df.Balance.describe()

In [ ]:
Q1 , Q3 = df.Balance.quantile([0.35, 0.85])
Q1, Q3

In [ ]:
df = df.loc[(df.Balance >=  Q1) & (df.Balance <= Q3)]
df

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

### using smote to oversample minority samples

In [ ]:
X = df.drop(['Exited'],axis = "columns")
y = df['Exited']

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy = 'minority')
X_sm, y_sm = smote.fit_sample(X,y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm,y_sm,test_size = 0.2, random_state = 42, stratify = y_sm)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()

In [ ]:
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [ ]:
X_train_sc.shape, y_train.shape

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(298, input_dim = 11, activation = 'relu')),
model.add(Dense(1,activation = 'sigmoid'))

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.fit(X_train_sc, y_train, epochs = 100)

In [ ]:
model.evaluate(X_test_sc, y_test)

In [ ]:
y_preds = model.predict(X_test_sc)

In [ ]:
predictions = []
for i in y_preds:
    if(i>0.5):
        predictions.append(1)
    else:
        predictions.append(0)

In [ ]:
from sklearn.metrics import classification_report 
print(classification_report(y_test, predictions))